# Auto Sampler script
## Elveflow Instruments
* OB1 pressure controller
* MUX Wire
* 

In [ ]:
# Import Elveflow library
import sys
from email.header import UTF8
SDK_HOME = 'C:/Users/T5810/Downloads/ESI_V3_04_01/SDK'
sys.path.append(SDK_HOME+'/DLL64/DLL64')#add the path of the library here
sys.path.append(SDK_HOME+'/Python_64')#add the path of the LoadElveflow.py

from ctypes import *

from array import array

from Elveflow64 import *

In [ ]:
# Initialize instruments

# OB1 and flow sensor initialization
Instr_ID=c_int32()
print("Instrument name and regulator types are hardcoded in the Python script")
#see User Guide to determine regulator types and NIMAX to determine the instrument name 
error=OB1_Initialization('01C9D9C3'.encode('ascii'),1,2,4,3,byref(Instr_ID)) 
#all functions will return error codes to help you to debug your code, for further information refer to User Guide
print('error:%d' % error)
print("OB1 ID: %d" % Instr_ID.value)

#add one digital flow sensor with water calibration (OB1 MK3+ only), all information to declare sensors are described in the User Guide
error=OB1_Add_Sens(Instr_ID, 1, 1, 1, 0, 7, 0)
#(CustomSens_Voltage_5_to_25 only works with CustomSensors and OB1 from 2020 and after)
print('error add digit flow sensor:%d' % error)


#
#Set the calibration type
#

Calib=(c_double*1000)()#always define array this way, calibration should have 1000 elements
repeat=True
while repeat==True:
    answer=input('select calibration type (default, load, new ) : ')
    #answer='default'#test purpose only
    Calib_path='C:\\Users\\Public\\Desktop\\Calibration\\Calib.txt'
    if answer=='default':
        error=Elveflow_Calibration_Default (byref(Calib),1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        repeat=False
    if answer=='load':
        error=Elveflow_Calibration_Load (Calib_path.encode('ascii'), byref(Calib), 1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        repeat=False
        
    if answer=='new':
        OB1_Calib (Instr_ID.value, Calib, 1000)
        #for i in range (0,1000):
            #print('[',i,']: ',Calib[i])
        error=Elveflow_Calibration_Save(Calib_path.encode('ascii'), byref(Calib), 1000)
        print ('calib saved in %s' % Calib_path.encode('ascii'))
        repeat=False